In [1]:
import numpy as np
import keras
import cv2
import pandas as pd
import tensorflow as tf
from keras import backend as K
from keras.models import Sequential
from keras.layers import Activation
from keras.layers import Dropout
from keras.layers import InputLayer
from keras.layers.core import Dense, Flatten
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import *
from matplotlib import pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import OneHotEncoder
import itertools
import matplotlib.pyplot as plt
import os
%matplotlib inline

Using TensorFlow backend.


In [2]:
file_path = 'D:\Admin\Documents\Year_4\AMLS\Assessment\dataset_AMLS_20-21\celeba\img'
labels_path = 'D:\Admin\Documents\Year_4\AMLS\Assessment\dataset_AMLS_20-21\celeba\labels.csv'

In [3]:
dftr = pd.read_csv(labels_path, skiprows=lambda x: x in range(1,5000,5))
dftr["smiling"] = dftr["smiling"].replace(to_replace=[-1], value=['Frown'])
dftr["smiling"] = dftr["smiling"].replace(to_replace=[1], value=['Smile'])
dftr["gender"] = dftr["gender"].replace(to_replace=[-1], value=['Female'])
dftr["gender"] = dftr["gender"].replace(to_replace=[1], value=['Male'])
one_hot_s = pd.get_dummies(dftr["smiling"])
one_hot_g = pd.get_dummies(dftr["gender"])
dftr = dftr.drop(columns=["gender", "smiling"])
dftr = dftr.join(one_hot_g)
dftr = dftr.join(one_hot_s)
datagen = ImageDataGenerator(rescale=1./255)
train_generator = datagen.flow_from_dataframe(
    dataframe=dftr, 
    directory=file_path,
    x_col="img_name", 
    y_col=["Male", "Female"], 
    class_mode="raw", 
    target_size=(55,45), 
    batch_size=100, 
    color_mode='grayscale', 
    interpolation='bicubic')

Found 4000 validated image filenames.


In [4]:
dfv = pd.read_csv(labels_path, skiprows=lambda x: x not in range(0,5000,5))
dfv["smiling"] = dfv["smiling"].replace(to_replace=[-1], value=['Frown'])
dfv["smiling"] = dfv["smiling"].replace(to_replace=[1], value=['Smile'])
dfv["gender"] = dfv["gender"].replace(to_replace=[-1], value=['Female'])
dfv["gender"] = dfv["gender"].replace(to_replace=[1], value=['Male'])
one_hot_s = pd.get_dummies(dfv["smiling"])
one_hot_g = pd.get_dummies(dfv["gender"])
dfv = dfv.drop(columns=["gender", "smiling"])
dfv = dfv.join(one_hot_g)
dfv = dfv.join(one_hot_s)
datagen = ImageDataGenerator(rescale=1./255)
test_generator = datagen.flow_from_dataframe(
    dataframe=dfv, 
    directory=file_path,
    x_col="img_name", 
    y_col=["Male", "Female"], 
    class_mode="raw", 
    target_size=(55,45), 
    batch_size=100, 
    color_mode='grayscale', 
    interpolation='bicubic')

Found 999 validated image filenames.


In [5]:
model = Sequential()
model.add(InputLayer(input_shape=(55,45,1)))
model.add(Conv2D(filters=96, kernel_size=(4,4), strides=4, activation="relu", padding="same"))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())
model.add(Conv2D(filters=256, kernel_size=(3,3), strides=1, activation="relu", padding="same"))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())
model.add(Conv2D(filters=256, kernel_size=(2,2), activation="relu", strides=1, padding="same"))
model.add(Flatten())
model.add(Dense(units=512, activation="relu"))
model.add(Dropout(0.2))
model.add(Dense(units=512, activation="relu"))
model.add(Dropout(0.2))
model.add(Dense(2, activation="softmax"))

In [6]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 14, 12, 96)        1632      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 7, 6, 96)          0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 7, 6, 96)          384       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 7, 6, 256)         221440    
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 3, 3, 256)         0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 3, 3, 256)         1024      
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 3, 3, 256)        

In [7]:
model.compile(Adam(lr=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

In [8]:
train_steps = train_generator.n//train_generator.batch_size
test_steps = test_generator.n//test_generator.batch_size

In [9]:
model.fit_generator(generator=train_generator, 
                    steps_per_epoch=train_steps,
                    validation_data=test_generator,
                    validation_steps=test_steps,
                    epochs=20)

Epoch 1/20
40/40 [==============================] - 8s 199ms/step - loss: 0.5820 - accuracy: 0.6895 - val_loss: 0.6881 - val_accuracy: 0.5656
Epoch 2/20
40/40 [==============================] - 9s 223ms/step - loss: 0.4083 - accuracy: 0.8167 - val_loss: 0.6763 - val_accuracy: 0.7430
Epoch 3/20
40/40 [==============================] - 8s 209ms/step - loss: 0.2933 - accuracy: 0.8845 - val_loss: 0.6908 - val_accuracy: 0.4705
Epoch 4/20
40/40 [==============================] - 8s 209ms/step - loss: 0.2107 - accuracy: 0.9225 - val_loss: 0.6430 - val_accuracy: 0.4716
Epoch 5/20
40/40 [==============================] - 8s 204ms/step - loss: 0.1539 - accuracy: 0.9413 - val_loss: 0.6331 - val_accuracy: 0.4805
Epoch 6/20
40/40 [==============================] - 8s 206ms/step - loss: 0.1094 - accuracy: 0.9613 - val_loss: 0.7699 - val_accuracy: 0.4661
Epoch 7/20
40/40 [==============================] - 8s 205ms/step - loss: 0.0619 - accuracy: 0.9845 - val_loss: 0.8795 - val_accuracy: 0.4828
Epoch 